## Символьные нграммы

Одним из начальных этапов в решении многих задач NLP является построение словаря. Получившийся словарь фиксируется и уже не изменяется на следующих этапах. Если нужен новый словарь - то нужно перестраивать все решение целиком. 

Например, в векторизации мешком слов словарь определяет размерность вектора и соответствие значений конкретным словам, а классификатор, который можно обучить на полученных векторах, подстраивается под эту размерность и порядок значений. Таким образом, нельзя просто взять и добавить какое-то слово в словарь и использовать уже имеющийся векторизатор и классификатор. 

В простых моделях и при небольшом количестве данных обучить все с нуля не трудно. Но на практике такие условия встречаются достаточно редко, поэтому словарь желательно собрать один раз и надолго.

Помимо очевидного решения использовать лемматизацию и большой корпус, есть еще одно, которое не такое очевидное, но все еще достаточно простое и эффективное - использовать вместо слов символьные нграммы.

Идея достаточно простая - вместо токенизации по пробелам, можно разделить текст на значимые кусочки слов (приставки, суффиксы, просто частотные сочетания и даже единичные символы). Чем короче сочетание символов, тем меньше вариантов возможно и к тому же в языках, как правило, есть устойчивые и невозможные сочетаний - другими словами словарь будет меньше. Также один кусочек может встречаться во многих словах и таким образом словарь меньшего по размеру объема будет представлять больший набор уникальных слов и будет обобщаться но новые слова. 

In [149]:
Image(url="https://miro.medium.com/max/1600/1*7_s0e2RuWz5_0GYwqQzFlQ.png",
     width=600, height=500)

Кажется, что в мешке слов, токенизация на символьные нграммы не применима - так как порядок размывается еще сильнее, но на практике все отлично работает. В sklearn это даже встроено в векторайзеры.

In [ ]:
!pip install eli5

In [98]:
import pandas as pd
import numpy as np

from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

from IPython.display import Image
from IPython.core.display import HTML 

Посмотрим, как можно по символьным нграммам предсказывать тональность сообщений в одноклассниках.

In [22]:
data = pd.read_csv('dataset_ok.csv')

In [25]:
data

,text,label
0,"наебалово века, для долбаёбов\n",INSULT
1,вся дума в таком же положении😁\n,NORMAL
2,а в каком месте массовое столкновение? шрайбик...,NORMAL
3,"значит ли это, что контроль за вывозом крупног...",NORMAL
4,вам не нужен щеночек? очень хорошие 🐶🥰\n,NORMAL
...,...,...
71982,"царствие небесное ,🙏, одноклассник,\n",NORMAL
71983,здоровье вам маленькие детки\n,NORMAL
71984,я тоже дочери покупала эффекта ноль это развод...,NORMAL
71985,почему он держит чубайса до сих пор?\n,NORMAL


In [24]:
train_texts, test_texts, train_labels, test_labels = train_test_split(data.text, data.label, 
                                                                      test_size=0.1, shuffle=True)

Чтобы перейти на уровень символов в векторайзере нужно указать параметр analyzer='char', а чтобы получались не отдельные символы, а нграммы - указать параметр ngram_range=(3, 3) (нижняя и верхняя граница длины нграмма, можно ставить любые, по умолчанию (1,1)

In [56]:
vectorizer = CountVectorizer(min_df=10, max_df=0.4, analyzer='char_wb', ngram_range=(5,5), max_features=2000)
X = vectorizer.fit_transform(train_texts)
X_test = vectorizer.transform(test_texts) 

y = np.array(train_labels)
y_test = np.array(test_labels)

По сравнению с предыдущим семинаром данных стало сильно больше. Даже простые классификаторы в sklearn не очень хорошо масштабируются на датасеты размером > 20k. Но можно использовать SGDClassifier - это линейный классификатор, которые обучается небольшими кусочками - получается быстрее и эффективнее. Если указать параметр loss='log', то это будет по сути LogisticRegression только в разы быстрее.

In [65]:
# вместо C тут alpha
# n_iter - задает максимальное количество проходов по датасету
# чем меньше, тем быстрее обучиться (но может недооучиться)
clf = SGDClassifier(loss="log", n_iter=30, alpha=0.0001, class_weight='balanced')

clf.fit(X, y)
preds = clf.predict(X_test)

print(classification_report(y_test, preds))

/Users/mnefedov/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


             precision    recall  f1-score   support

     INSULT       0.80      0.63      0.71       903
     NORMAL       0.94      0.95      0.95      6041
  OBSCENITY       0.23      0.39      0.29        70
     THREAT       0.58      0.70      0.63       185

avg / total       0.90      0.90      0.90      7199



### Кросс-валидация

Мы оценили классификатор на отложенной выборке. Это нормальное решение, но есть решение получше - кросс-валидация. Это по сути несколько разбиений на трейн и тест с обучением и оцениванием на каждом разбиении. Если данных не очень много или есть дисбаланс классов, то от разбиения может сильно зависеть качество классификатора (в трейн могут попасть простые примеры, а в тест сложные - тогда классификатор недооучиться и метрики будут ниже, чем могли бы быть и наоборот).

Кросс-валидация позволяет сгладить значимость разбиения и получить более точное представление о качестве классификатора.

In [148]:
Image(url="https://scikit-learn.org/stable/_images/grid_search_cross_validation.png",
     width=600, height=500)

В sklearn есть много инстурментов для кросс-валидации, давайте пока рассмотрим самый простой - cross_val - это функция, в которую можно подать все данные, классификатор, а остальное она сделаем сама.

In [93]:
from sklearn.model_selection import cross_val_score

In [94]:
vectorizer = TfidfVectorizer(min_df=5, max_df=0.4, analyzer='char', ngram_range=(3,3), max_features=2000)

X = vectorizer.fit_transform(data.text)
y = np.array(data.label)


In [30]:
clf = SGDClassifier(loss="log", n_iter=50)

Указываем какую метрику хотим посчитать

In [ ]:
cross_val_score(clf, X, y, scoring="f1_micro")

In [34]:
cross_val_score(clf, X, y, scoring="f1_micro")

/Users/mnefedov/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/Users/mnefedov/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/Users/mnefedov/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


array([0.90603384, 0.90810586, 0.90931066])

В обоих случаях мы считали f1 меру, но результаты сильно разные. Почему так? Потому что можно по-разному усреднять метрики - если усреднять уже посчитанную f1 меру по каждому классу, то это макро f1 мера, а если для каждого класса посчитать true positives, true negatives, false positives, просуммировать их и посчитать 1 общую f1 меру - то это микро среднее.

Макро f1 мера уравнивает классы, так как количество примеров не влияет на усредненную оценки. В микро усреднении на результат влияет количество примеров каждого класса, поэтому если есть перекос в сторону хорошо определяемого класса, то метрики будут завышенные (как в classification_report выше, там есть класс с очень плохой f1 мерой, но общая метрика все равно 90 %)


### Анализ признаков

 Иногда возникает потребность понять, на какие признаки опирается классификатор при принятии решений. Также анализ значимости признаков может помочь в улучшении модели.

Есть библиотека eli5, которая упрощает анализ признаков

In [66]:
import eli5

Посмотрим какие признаки (символьные нграммы) наиболее характерны для каждого из классов.

In [76]:
# видно, что всякие нецензурные основы являются триггером класса INSULT
# и антитриггером нормального класса
eli5.show_weights(clf, top=10, feature_names=vectorizer.get_feature_names())

Еще одна полезная функция - объяснение предсказания для конкретного текста

In [117]:
eli5.show_prediction(clf, data.loc[31, 'text'], vec=vectorizer)

# Готовые subword токенайзеры

Символьные нграммы в sklearn можно использовать только если нужно векторизовать мешком слов. Если нужно сохранить порядок, то лучше использовать специализированные библиотеки для subword токенизации. Например, библиотеку tokenizers от [HuggingFace](https://huggingface.co/). Он к тому же намного быстрее.

In [78]:
!pip install tokenizers

     |████████████████████████████████| 2.0 MB 1.7 MB/s eta 0:00:01
You should consider upgrading via the '/Users/mnefedov/.pyenv/versions/3.6.5/bin/python3.6 -m pip install --upgrade pip' command.


Для обучения нужно сохранить тексты в файл

In [139]:
from tokenizers import CharBPETokenizer, Tokenizer

In [80]:
data['text'].to_csv('corpus.txt', index=None)

Обучаем CharBPE токенизатор (BPE - Byte-Pair-Encoding - название такого метода токенизации, вот тут можно почитать подробнее - https://towardsdatascience.com/byte-pair-encoding-the-dark-horse-of-modern-nlp-eb36c7df4f10)

In [140]:
tok_sub = CharBPETokenizer()
tok_sub.train('corpus.txt', vocab_size=2000, min_frequency=10,)

Сохранение и загрузка.

In [141]:
tok_sub.save('2k')

In [142]:
tok_sub = Tokenizer.from_file("2k")

Токенизировать текст можно вот так.

In [144]:
tok_sub.encode(data.loc[0, 'text']).tokens

['на',
 'еба',
 'ло',
 'во</w>',
 'ве',
 'ка</w>',
 ',</w>',
 'для</w>',
 'дол',
 'ба',
 'ё',
 'бо',
 'в</w>']

Для NLP задач обычно нужны не токены, а индексы

In [143]:
tok_sub.encode(data.loc[0, 'text']).ids

[1337, 1898, 1361, 1547, 1362, 1390, 1159, 1534, 1548, 1409, 129, 1366, 758]

Словарь можно посмотреть вот так.

In [145]:
tok_sub.get_vocab()

{'опе': 1879,
 '🇿': 287,
 '🎹': 383,
 '7': 22,
 '🚞</w>': 979,
 '🏼</w>': 914,
 '⏰': 195,
 '⛪': 228,
 '\U0001f91f</w>': 1061,
 '♐</w>': 1145,
 '\U0001f971': 721,
 '🏢': 397,
 '°': 69,
 '🍲': 351,
 'ё': 129,
 '👜</w>': 1287,
 '1</w>': 787,
 'луч': 1695,
 '♐': 212,
 '👵': 469,
 'ав': 1699,
 'ение</w>': 1541,
 '🚂</w>': 920,
 'щ': 122,
 'один</w>': 1952,
 '👼</w>': 1181,
 '\U0001f975': 724,
 'если</w>': 1498,
 '💟</w>': 1193,
 '🏖': 393,
 '\U0001f9b7': 734,
 '😂': 573,
 'م</w>': 1258,
 'го': 1369,
 'ө': 139,
 'ро': 1334,
 '🌸': 313,
 '☄': 198,
 'ья</w>': 1503,
 '🌇': 294,
 '🥞': 716,
 'фа': 1821,
 '🦊</w>': 1102,
 'раз</w>': 1789,
 '🇳': 281,
 '🛩': 668,
 'боль': 1529,
 '⠀</w>': 944,
 'ная</w>': 1485,
 '{': 61,
 '🌠': 305,
 '👛': 452,
 '🎵</w>': 1232,
 'ка</w>': 1390,
 'вает</w>': 1774,
 '♀': 209,
 '😱': 620,
 '💙</w>': 1205,
 '🐘</w>': 1269,
 '🌺': 315,
 'его</w>': 1509,
 '📷': 537,
 '🗣': 568,
 '📹</w>': 1325,
 '👯': 467,
 'па': 1396,
 'дру': 1510,
 'мо</w>': 1818,
 '}': 63,
 '💫': 510,
 '😏</w>': 911,
 '89': 1930,
 